# This notebook will go through a Linear Regression Example using SPARK

Linear Regression Example

Creating a spark session

In [1]:
from pyspark.sql import SparkSession 

In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
# Use Spark to read in the Ecommerce Customers csv file.
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lnr = LinearRegression(featuresCol ='features', labelCol ='label',
                       predictionCol ='prediction',)

In [7]:
lnrModel= lnr.fit(training)

In [8]:
lnrModel.coefficients
# Regression co-efficents

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lnrModel.intercept
# Intercept

0.14228558260358093

In [10]:
training_summary = lnrModel.summary

In [11]:
training_summary.rootMeanSquaredError
#training error

10.16309157133015

Did not split into train/test datasets previously, now do

In [12]:
total_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [13]:
train_d, test_d = total_data.randomSplit([0.75,0.25])

In [14]:
train_d.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
+----------

In [15]:
train_d.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 362|
|   mean|-0.07174704883829373|
| stddev|  10.274176092041982|
|    min| -28.046018037776633|
|    max|   27.78383192005107|
+-------+--------------------+



In [16]:
test_d.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                139|
|   mean|  1.112760658986705|
| stddev| 10.419380076532194|
|    min|-28.571478869743427|
|    max| 27.111027963108548|
+-------+-------------------+



In [18]:
true_model = lnr.fit(train_d)

In [19]:
test_results = true_model.evaluate(test_d)

Can now grab various stats

In [20]:
test_results.meanSquaredError

110.24151402609556

In [21]:
test_results.degreesOfFreedom

128

In [22]:
test_results.rootMeanSquaredError

10.499595898228444

In [24]:
unseen_data = test_d.select('features')

In [25]:
unseen_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [26]:
predictions = true_model.transform(unseen_data)

In [27]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.7753956720305697|
|(10,[0,1,2,3,4,5,...|    4.57749634838486|
|(10,[0,1,2,3,4,5,...|   1.470474320911259|
|(10,[0,1,2,3,4,5,...| 0.16501088855985963|
|(10,[0,1,2,3,4,5,...|   0.382631122007241|
|(10,[0,1,2,3,4,5,...|  -2.244641180439401|
|(10,[0,1,2,3,4,5,...|-0.19767851034218237|
|(10,[0,1,2,3,4,5,...|   0.554128350217123|
|(10,[0,1,2,3,4,5,...|  0.9579936083127973|
|(10,[0,1,2,3,4,5,...|  1.4860809917672964|
|(10,[0,1,2,3,4,5,...| -0.8918033981883211|
|(10,[0,1,2,3,4,5,...| -1.7770536285258844|
|(10,[0,1,2,3,4,5,...|  1.0179778981790013|
|(10,[0,1,2,3,4,5,...|  -2.531844537128003|
|(10,[0,1,2,3,4,5,...| -1.1290207808854191|
|(10,[0,1,2,3,4,5,...|  2.7949769958129265|
|(10,[0,1,2,3,4,5,...|   0.942338521342674|
|(10,[0,1,2,3,4,5,...| -1.6499765117252692|
|(10,[0,1,2,3,4,5,...| -1.4729374075202453|
|(10,[0,1,2,3,4,5,...|0.00635926